In [1]:
%run Common.ipynb

C:\Source\CSIRO\HealthyHabitatAI\.env
healthyhabitatanimals
healthyhabitatparragrass
ghNvkPl4DYLUUJh55y4pSg3BVV8TtPcHcX96P/zaDPXr9uuREdkGSCaao59d6xfZOfg4l9um1qfBXuNM9VGTTg==
jpEJGrFNi6PNLGOOD62rFBxjHGKTR4+DcppQNQQXi3P/GFmINKfqf8EZJoo73doIAnntv03ZSUsAC442IRR87Q==
processed
processed
f8fc2cd4db6b45a59d1ddd69a8685ddd
4610573e-864f-4f53-9f6a-c61ce520ed5a
5aeb092f-e91d-400f-bd88-e9516b108f20


In [2]:
import numpy as np
import matplotlib.pyplot as plt

from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry
from PIL import Image

In [3]:
%matplotlib inline

### Custom Vision

In [4]:
ENDPOINT = 'https://australiaeast.api.cognitive.microsoft.com'

trainer = CustomVisionTrainingClient(custom_vision_training_key, endpoint=ENDPOINT)

projects = trainer.get_projects()

### Blob Storage

#### Magpie Geese Containers - *Site-Season*

In [6]:
magpie_geese_containers = blob_service_list_containers(healthy_habitat_animals_storage_name, healthy_habitat_animals_storage_key)

for container_name in magpie_geese_containers:
    print(container_name)

cannon-hill-kunumeleng
cannon-hill-wurrkeng
jabiru-dreaming-kunumeleng
jabiru-dreaming-wurrkeng
ubir-kunumeleng
ubir-wurrkeng


### Create regions

#### Magpie Geese

In [7]:
for container_name in magpie_geese_containers:
    if container_name == 'ubir-wurrkeng':
        blobs = blob_service_list_blobs(healthy_habitat_animals_storage_name, healthy_habitat_animals_storage_key, container_name)
        
        folders = set([blob.split('/')[0] for blob in blobs])
        
        for folder in folders:
            interim_folder_path = os.path.join(Path(os.getcwd()).parent, 'data', 'interim', 'animal', container_name, folder)
            if not os.path.exists(interim_folder_path):
                os.makedirs(interim_folder_path)

        for blob_name in blobs:
            folder = blob_name.split('/')[0]
            file = blob_name.split('/')[1]
            
            filter = "PartitionKey eq '{0}' and RowKey eq '{1}'".format(folder, file)
            
            entities = table_service_query_entities(healthy_habitat_animals_storage_name, healthy_habitat_animals_storage_key, healthy_habitat_animals_processed_table_name, filter)
            
            count = len([entity for entity in entities])

            if count == 0:
                print('Processing {0}'.format(blob_name))
                file_path = os.path.join(interim_folder_path, file)

                if not os.path.exists(file_path):
                    blob_service_get_blob_to_path(healthy_habitat_animals_storage_name, healthy_habitat_animals_storage_key, container_name, blob_name, file_path)

                image = np.array(Image.open(file_path))
                image_shape = image.shape

                height = 228
                width = 304

                count = 0
                
                for y in range(0, image_shape[0], height):
                    for x in range(0, image_shape[1], width):
                        region = image[y:y + height, x:x + width]

                        file_name = '{0}_Region_{1}.jpg'.format(file.split('.')[0], count)

                        buffer = io.BytesIO()
                        Image.fromarray(region).save(buffer, format='JPEG')        

                        images = []
                            
                        images.append(ImageFileCreateEntry(name=file_name, contents=buffer.getvalue()))

                        result = trainer.create_images_from_files(custom_vision_magpie_geese_project_id, images=images)

                        if not result.is_batch_successful:
                            for image_create_result in result.images:
                                print("Image status: ", image_create_result.status)
                                
                        count += 1

                entity = {'PartitionKey': folder, 'RowKey': file}
                table_service_insert(healthy_habitat_animals_storage_name, healthy_habitat_animals_storage_key, healthy_habitat_animals_processed_table_name, entity)
                
                os.remove(file_path)
                print(count)

Processing 2019-08-13-1700/DJI_0013.JPG
288
Processing 2019-08-13-1700/DJI_0014.JPG
288
Processing 2019-08-13-1700/DJI_0015.JPG
288
Processing 2019-08-13-1700/DJI_0016.JPG


WARNING - Retrying (Retry(total=2, connect=3, read=3, redirect=None, status=None)) after connection broken by 'SSLError(SSLError("bad handshake: SysCallError(10054, 'WSAECONNRESET')",),)': /customvision/v3.0/training/projects/5aeb092f-e91d-400f-bd88-e9516b108f20/images/files
WARNING - Retrying (Retry(total=1, connect=2, read=3, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001891F8FFBE0>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',)': /customvision/v3.0/training/projects/5aeb092f-e91d-400f-bd88-e9516b108f20/images/files
WARNING - Retrying (Retry(total=0, connect=1, read=3, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001891F8FF668>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',)': /customvision/v3.0/training/projects/5aeb092f-e91d-400f-bd88-e9516b108f20

ClientRequestError: Error occurred in request., ConnectionError: HTTPSConnectionPool(host='australiaeast.api.cognitive.microsoft.com', port=443): Max retries exceeded with url: /customvision/v3.0/training/projects/5aeb092f-e91d-400f-bd88-e9516b108f20/images/files (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x000001891E908400>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))

In [ ]:
tagged_image_count = trainer.get_tagged_image_count(custom_vision_magpie_geese_project_id)
print('Tagged image count:', tagged_image_count)

untagged_image_count = trainer.get_untagged_image_count(custom_vision_magpie_geese_project_id)
print('Untagged image count:', untagged_image_count)

print('Image count', tagged_image_count + untagged_image_count)